# Computing the bicharacter of huge spaces of polynomials

*TODO* : Add intro

In [1]:
%display latex
%runfile compute_character.py

## Code

Computation of the Vandermonde determinant associated to the diagram $\gamma$, where $\gamma$ is a partation (Partition) or a list of cells (Diagram). 

Functions used to get the lists of operators by degrees for all the non inert variables of the given polynomial ring. 

Projection on the isotypic components; take as parameter a subspace and an integer or a partition. 

Create the polarized space given by the input operators.

Compute the image of S by the given operators.

Computation of the character from a subspace already projected on the isotypic components. 

Factorisation of the tensorial products by represention of $S_n$. And computation of the dimensions. 

## Examples

### Partition 21

Computation of the Vandermonde determinant $\mathcal{V}_\lambda$ associated to the partition $\lambda = 21$.

In [2]:
v = vandermonde(Partition([2,1]))
v

-x01*theta00 + x02*theta00 + x00*theta01 - x02*theta01 - x00*theta02 + x01*theta02

Space generated by $\mathcal{V}_\lambda$ and its partials derivatives.

In [3]:
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
V1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
V1.basis()

{(0,): (theta01 - theta02, theta00 - theta02),
 (1,): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

Projection on the isotypic components.

In [4]:
V1_iso = IsotypicComponent(V1, 3)
for subspace in V1_iso.values():
    show(subspace.basis())

{((1,),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0,), (2, 1)): (-theta00 + theta02,)}

We add another set of variables and we polarize.

In [5]:
op_pol = polarization_operators(2)
V1_pol = PolarizedSpace(V1_iso, op_pol)
for subspace in V1_pol.values():
    show(subspace.basis())

{((0, 1),
  (1,
   1,
   1)): (-x11*theta00 + x12*theta00 + x10*theta01 - x12*theta01 - x10*theta02 + x11*theta02,),
 ((1, 0),
  (1,
   1,
   1)): (x01*theta00 - x02*theta00 - x00*theta01 + x02*theta01 + x00*theta02 - x01*theta02,)}

{((0, 0), (2, 1)): (theta00 - theta02,)}

Computation of the bicharacter.

In [6]:
character(V1_pol)

s[] # s[2, 1] + s[1] # s[1, 1, 1]

### Diagram (0,1,3)

In [7]:
v = vandermonde(Diagram([(0,0),(1,0),(3,0)]))
v.factor()

(x01 - x02) * (-x00 + x01) * (x00 - x02) * (x00 + x01 + x02)

We create the subspace generated by $\mathcal{V}_\gamma$ and its partial derivatives. We can add the Steenrod operators without any change in the result.

In [8]:
generator = {v.multidegree() : [v]}
list_op = merge(merge(partial_derivatives(v.parent()), steenrod_operators(1, 1)), steenrod_operators(1, 2))

In [9]:
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
W1.basis()

{(0,): (1,),
 (1,): (x00, x01, x02),
 (2,): (x00^2 - x02^2, x01^2 - x02^2, -x00*x01 + x00*x02, -x00*x01 + x01*x02),
 (3,): (1/3*x00^3 - 1/3*x01^3 - x00*x02^2 + x01*x02^2,
  -1/3*x00^3 + x00*x01^2 - x01^2*x02 + 1/3*x02^3,
  -x00^2*x01 + 1/3*x01^3 + x00^2*x02 - 1/3*x02^3),
 (4,): (-x00^3*x01 + x00*x01^3 + x00^3*x02 - x01^3*x02 - x00*x02^3 + x01*x02^3,)}

In [10]:
character(IsotypicComponent(W1, 3))

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Apply $\sum_i \partial_{x_i}$ and $\sum_i x_i \partial_{x_i}^2$ à $\mathcal{V}_{\gamma}$. 

In [11]:
v.symmetric_derivative((1,))

-3*x00^2*x01 + 3*x00*x01^2 + 3*x00^2*x02 - 3*x01^2*x02 - 3*x00*x02^2 + 3*x01*x02^2

In [12]:
v.steenrod_op(0, 2)

-6*x00^2*x01 + 6*x00*x01^2 + 6*x00^2*x02 - 6*x01^2*x02 - 6*x00*x02^2 + 6*x01*x02^2

Polarization

In [16]:
r=2
op_pol = polarization_operators(r)
W2 = PolarizedSpace(IsotypicComponent(W1, 3), op_pol)
character(W2)

s[] # s[3] + s[1] # s[2, 1] + s[1] # s[3] + 2*s[2] # s[2, 1] + s[3] # s[1, 1, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

In [18]:
list_degrees = [tuple(k1 if j==i1 else 0 for j in range(r)) 
                for k1 in range(1, 3+1) for i1 in range(0, r)]
#list_degrees += [(1,1),(1,2),(1,3),(2,1),(2,2),(2,3),(3,1),(3,2),(3,3)]
sym_diff = symmetric_derivatives(list_degrees)
#sym_diff = merge(merge(symmetric_derivatives(P2, list_degrees), 
#                       steenrod_operators(P2, 1)), steenrod_operators(P2, 2))
character(Range(W2, sym_diff))

s[] # s[3] + s[1] # s[2, 1] + s[2] # s[2, 1] + s[3] # s[1, 1, 1]

In [19]:
charac = character(W2) - character(Range(W2, sym_diff))
charac

s[1] # s[3] + s[2] # s[2, 1] + s[3] # s[2, 1] + s[4] # s[1, 1, 1]

Or in terms of monomial symmetric functions.

In [20]:
character(W2, left_basis=m) - character(Range(W2, sym_diff), left_basis=m)

m[1] # s[3] + m[1, 1] # s[2, 1] + m[2] # s[2, 1] + m[2, 1] # s[2, 1] + m[2, 2] # s[1, 1, 1] + m[3] # s[2, 1] + m[3, 1] # s[1, 1, 1] + m[4] # s[1, 1, 1]

We factorise the character in terms of representations of $S_3$.

In [21]:
factorise(charac, 3)

[1, 1, 1]


s[4]

[3]


s[1]

[2, 1]


s[2] + s[3]

### Partition 311

In [22]:
mu = Partition([3,1,1])
v = vandermonde(mu)
n = mu.size()
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(n-1)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
character(W2)

s[] # s[3, 1, 1] + s[1] # s[2, 1, 1, 1] + s[1] # s[2, 2, 1] + s[2] # s[2, 1, 1, 1] + s[3] # s[1, 1, 1, 1, 1]

### Others cases and examples

Comparison between ... 

In [23]:
n = 4
gamma = Partition([2,2])
v = vandermonde(gamma)
show(v)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
show(character(IsotypicComponent(W1, n)))

r = 3
op_pol = polarization_operators(r)
W1 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
character(W1)

x00*x02*theta00*theta01 - x01*x02*theta00*theta01 - x00*x03*theta00*theta01 + x01*x03*theta00*theta01 - x00*x01*theta00*theta02 + x01*x02*theta00*theta02 + x00*x03*theta00*theta02 - x02*x03*theta00*theta02 + x00*x01*theta01*theta02 - x00*x02*theta01*theta02 - x01*x03*theta01*theta02 + x02*x03*theta01*theta02 + x00*x01*theta00*theta03 - x00*x02*theta00*theta03 - x01*x03*theta00*theta03 + x02*x03*theta00*theta03 - x00*x01*theta01*theta03 + x01*x02*theta01*theta03 + x00*x03*theta01*theta03 - x02*x03*theta01*theta03 + x00*x02*theta02*theta03 - x01*x02*theta02*theta03 - x00*x03*theta02*theta03 + x01*x03*theta02*theta03

s[] # s[2, 2] + s[1] # s[2, 1, 1] + s[2] # s[1, 1, 1, 1]

s[] # s[2, 2] + s[1] # s[2, 1, 1] + s[2] # s[1, 1, 1, 1]

In [24]:
n = 5
gamma = Partition([2,2,1])
v = vandermonde(gamma)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)
show(character(IsotypicComponent(W1, n)))

r = 4
op_pol = polarization_operators(r)
W1 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
character(W1)

s[] # s[2, 2, 1] + s[1] # s[2, 1, 1, 1] + s[2] # s[1, 1, 1, 1, 1]

s[] # s[2, 2, 1] + s[1] # s[2, 1, 1, 1] + s[2] # s[1, 1, 1, 1, 1]

*NOTE : add a bigger exemple with computation time + exemples of the classical case*

### Compare computations with and without optimisations

First, let compare bases on a small example, without and then with the use of antisymmetries.

In [25]:
mu = Partition([2,1,1])
n = mu.size()
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(3)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
for subspace in W2.values():
    show(subspace.basis())

s[] # s[2, 1, 1] + s[1] # s[1, 1, 1, 1]

{((0, 0, 0),
  (2,
   1,
   1)): (theta00^2*theta02 - theta00*theta02^2 - theta00^2*theta03 + theta02^2*theta03 + theta00*theta03^2 - theta02*theta03^2,)}

{((0, 0, 1),
  (1,
   1,
   1,
   1)): (-x22*theta00^2*theta01 + x23*theta00^2*theta01 + x22*theta00*theta01^2 - x23*theta00*theta01^2 + x21*theta00^2*theta02 - x23*theta00^2*theta02 - x20*theta01^2*theta02 + x23*theta01^2*theta02 - x21*theta00*theta02^2 + x23*theta00*theta02^2 + x20*theta01*theta02^2 - x23*theta01*theta02^2 - x21*theta00^2*theta03 + x22*theta00^2*theta03 + x20*theta01^2*theta03 - x22*theta01^2*theta03 - x20*theta02^2*theta03 + x21*theta02^2*theta03 + x21*theta00*theta03^2 - x22*theta00*theta03^2 - x20*theta01*theta03^2 + x22*theta01*theta03^2 + x20*theta02*theta03^2 - x21*theta02*theta03^2,),
 ((0, 1, 0),
  (1,
   1,
   1,
   1)): (-x12*theta00^2*theta01 + x13*theta00^2*theta01 + x12*theta00*theta01^2 - x13*theta00*theta01^2 + x11*theta00^2*theta02 - x13*theta00^2*theta02 - x10*theta01^2*theta02 + x13*theta01^2*theta02 - x11*theta00*theta02^2 + x13*theta00*theta02^2 + x10*theta01*theta02^2 - x13*theta01*theta02^2 - x11*theta00^2*theta03 + x12*theta00^2*theta03 + x10*theta01^2*theta03 - x12*theta01^2*theta03 - x10*theta02^2*theta03 + x11*theta02^2*theta03 + x11*theta00*theta03^2 - x12*theta00*theta03^2 - x10*theta01*theta03^2 + x12*theta01*theta03^2 + x10*theta02*theta03^2 - x11*theta02*theta03^2,),
 ((1, 0, 0),
  (1,
   1,
   1,
   1)): (-x02*theta00^2*theta01 + x03*theta00^2*theta01 + x02*theta00*theta01^2 - x03*theta00*theta01^2 + x01*theta00^2*theta02 - x03*theta00^2*theta02 - x00*theta01^2*theta02 + x03*theta01^2*theta02 - x01*theta00*theta02^2 + x03*theta00*theta02^2 + x00*theta01*theta02^2 - x03*theta01*theta02^2 - x01*theta00^2*theta03 + x02*theta00^2*theta03 + x00*theta01^2*theta03 - x02*theta01^2*theta03 - x00*theta02^2*theta03 + x01*theta02^2*theta03 + x01*theta00*theta03^2 - x02*theta00*theta03^2 - x00*theta01*theta03^2 + x02*theta01*theta03^2 + x00*theta02*theta03^2 - x01*theta02*theta03^2,)}

Compare computation time with or without antisymmetries on a bigger example.

In [26]:
import datetime

In [27]:
t1 = datetime.datetime.now()
mu = Partition([3,2,1])
n = mu.size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
t2 = datetime.datetime.now()
show(t2-t1)

KeyboardInterrupt: 

In [ ]:
t1 = datetime.datetime.now()
mu = Partition([3,2,1])
n = mu,size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, use_row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol)
show(character(W2, row_symmetry="permutation"))
t2 = datetime.datetime.now()
show(t2-t1)

Let now compare bases on a small example, without and then with the use of symmetries on rows.

In [ ]:
mu = Partition([2,1,1])
n = mu,size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r)
W2 = PolarizedSpace(IsotypicComponent(W1, n), op_pol)
show(character(W2))
for subsapce in W2.values():
    show(subspace.basis())

In [ ]:
mu = Partition([2,1,1])
n = mu,size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, use_row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=False), op_pol)
show(character(W2, row_symmetry="permutation"))
for subsapce in W2.values():
    show(subspace.basis())

Add also antisymmetries.

In [ ]:
mu = Partition([2,1,1])
n = mu,size()
r = n-1
v = vandermonde(mu)
deg_v = v.degree()
generator = {v.multidegree() : [v]}
list_op = partial_derivatives(v.parent())
W1 = Subspace(generators=generator, operators=list_op, add_degrees=add_degree)

op_pol = polarization_operators(r, use_row_symmetry="permutation")
W2 = PolarizedSpace(IsotypicComponent(W1, n, use_antisymmetry=True), op_pol)
show(character(W2, row_symmetry="permutation"))
for subsapce in W2.values():
    show(subspace.basis())

Compare computation time with or without symmetries on row on a bigger example.

In [ ]:
import datetime

In [ ]:
t1 = datetime.datetime.now()
compute_character(Partition([3,2]), use_antisymmetry=False, row_symmetry=None)
t2 = datetime.datetime.now()
show(t2-t1)

In [ ]:
t1 = datetime.datetime.now()
compute_character(Partition([3,2]), use_antisymmetry=False, row_symmetry="permutation")
t2 = datetime.datetime.now()
show(t2-t1)

In [ ]:
t1 = datetime.datetime.now()
compute_character(Partition([3,2]), use_antisymmetry=True, row_symmetry=None)
t2 = datetime.datetime.now()
show(t2-t1)

In [ ]:
t1 = datetime.datetime.now()
compute_character(Partition([3,2]), use_antisymmetry=True, row_symmetry="permutation")
t2 = datetime.datetime.now()
show(t2-t1)